In [ ]:
from torch_geometric.data import DataLoader
import torch.nn.functional as F
import torch, data_loader
from model import protein_feature_extractor, predictor, go_feature_extractor, att
from torch.autograd import Variable
from sklearn import metrics
import argparse, utils, os, evaluation_matrix, pickle, warnings, metric
import numpy as np
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
DATA_PATH = '/home/nas2/biod/zq/ZQ/Protein Function Prediction/graph_go/dataset/'
MODEL_PATH = "/home/nas2/biod/zq/ZQ/Protein Function Prediction/BIBM_journal/model_checkpoint/A_model_copy5/"
from tqdm import tqdm

In [2]:
batch_size = 64
epoches = 50
learning_rate = 0.0001

In [3]:
X_test = data_loader.Protein_Gnn_data(root = DATA_PATH + 'data/seq_features/', chain_list = DATA_PATH + "data/data_splits/test.pdb.txt")
test_loader = DataLoader(X_test, batch_size=batch_size, shuffle=True)
go_graph = utils.pickle_load(DATA_PATH+"PDB-cdhit/go_graph.pickle").to(device)

/home/zhangqiang/anaconda3/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [4]:
ont = ["all","bp","mf", "cc"]
Protein_feature_extraction = protein_feature_extractor.Protein_feature_extraction()
Label_feature_extraction = go_feature_extractor.Label_feature_extraction(ont[0])
Protein_Label_att = att.Protein_Label_att()
GnnPF_Model = predictor.information_exchange_2()
model = predictor.TotalModel(Protein_feature_extraction, Label_feature_extraction, Protein_Label_att, GnnPF_Model)
model = model.to(device)

In [8]:
def predict(loader, model):
    model.eval()
    y_pred_all = []
    y_true_all = []
    with torch.no_grad():
        for data in tqdm(loader):
            esm_rep, seq, contact, pssm, seq_embed = data.x.to(device), data.seq.T.unsqueeze(0).to(device), data.edge_index.to(device), data.pssm.T.unsqueeze(0).to(device), data.seq_embed.to(device)
            esm_rep = esm_rep.float()
            label = data.label.float()
            batch_idx = data.batch.to(device)
            model_pred = model(seq = seq, edge_index = contact, esm_token = esm_rep, esm_representation=seq_embed, batch = batch_idx, label_relationship = go_graph).to(torch.float32).detach()
            y_pred_all.append(model_pred)
            y_true_all.append(label)
        y_pred_all = torch.cat(y_pred_all, dim=0).cpu()
        y_true_all = torch.cat(y_true_all, dim=0)
        test_loss = loss_function(y_pred_all, y_true_all)
    return test_loss, y_pred_all, y_true_all

In [ ]:
best_model, optim, current_epoch, min_val_loss = utils.load_ckp(ckp_best, model, optim, device = device)
test_loss, y_pred_all, y_true_all = predict(test_loader, best_model)

In [ ]:
# evaluation and result save
result3 = metric.count_evaluation_TALE(y_pred_all, y_true_all)
print("mf_Fmax:%.3f mf_AUPRC:%.3f bp_Fmax:%.3f bp_AUPRC:%.3f cc_Fmax:%.3f cc_AUPRC:%.3f" %(result3[0],result3[1],result3[2], result3[3], result3[4], result3[5]))